<a href="https://colab.research.google.com/github/sedatoner/Motion-tracking-and-logging-with-Raspberry-Pi-and-Camera/blob/main/motion_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import datetime
import os
import csv
import imutils

# Klasörleri oluştur
os.makedirs("captures", exist_ok=True)
os.makedirs("logs", exist_ok=True)

# Log dosyası
log_path = "logs/motion_log.csv"
if not os.path.exists(log_path):
    with open(log_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Image Path"])

# Kamera başlat
cam = cv2.VideoCapture(0)
first_frame = None

while True:
    ret, frame = cam.read()
    if not ret:
        break

    frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    if first_frame is None:
        first_frame = gray
        continue

    delta = cv2.absdiff(first_frame, gray)
    thresh = cv2.threshold(delta, 25, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    motion_detected = False

    for contour in contours:
        if cv2.contourArea(contour) < 800:
            continue
        motion_detected = True
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    if motion_detected:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        img_path = f"captures/motion_{timestamp}.jpg"
        cv2.imwrite(img_path, frame)

        with open(log_path, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([timestamp, img_path])

        print(f"[{timestamp}] Motion detected! Image saved.")

    cv2.imshow("Live Feed", frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()
